<a href="https://colab.research.google.com/github/drustantmetar/DataFrame1/blob/main/Spark_DataFrame.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

spark data frame
**bold text**

In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [8]:
#sc: SparkContext= SparkSession\
spark = SparkSession\
.builder\
.appName("Spark-SQL")\
.master("local")\
.getOrCreate()

In [31]:
moviesdf=spark.read.option("header","true").option("inferschema","true").format("csv").load("/content/movies.csv")
ratingsdf=spark.read.format("csv").option("header","true").option("inferschema","true").load("/content/ratings.csv")

In [33]:
moviesdf.printSchema()
ratingsdf.printSchema()

root
 |-- movieId: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)



In [34]:
moviesdf.select("movieId","title").show(5)

+-------+--------------------+
|movieId|               title|
+-------+--------------------+
|      1|    Toy Story (1995)|
|      2|      Jumanji (1995)|
|      3|Grumpier Old Men ...|
|      4|Waiting to Exhale...|
|      5|Father of the Bri...|
+-------+--------------------+
only showing top 5 rows



In [35]:
ratingsdf.filter("rating<3.0").show(10)

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     2|    891|   2.0|974820969|
|     2|   1969|   2.0|974820969|
|     2|   1970|   2.0|974820969|
|     2|   1971|   2.0|974820943|
|     2|   1972|   2.0|974820969|
|     2|   1986|   2.0|974820969|
|     2|   2291|   2.0|974820748|
|     2|   2791|   2.0|974820659|
|     3|    173|   2.0|945176099|
|     3|    512|   2.0|945176005|
+------+-------+------+---------+
only showing top 10 rows



In [38]:
ratingsdf.groupBy("rating").count().orderBy("rating").show()

+------+-------+
|rating|  count|
+------+-------+
|   0.5| 162322|
|   1.0| 455078|
|   1.5| 186839|
|   2.0| 960915|
|   2.5| 595149|
|   3.0|2871144|
|   3.5|1479467|
|   4.0|3723912|
|   4.5|1030401|
|   5.0|1941520|
+------+-------+



In [40]:
ratingsdf.groupBy("rating").count().orderBy(col("rating").desc()).show()
#ratingsdf.groupBy("rating").count().orderBy(ratingsdf.rating.desc()).show()

+------+-------+
|rating|  count|
+------+-------+
|   5.0|1941520|
|   4.5|1030401|
|   4.0|3723912|
|   3.5|1479467|
|   3.0|2871144|
|   2.5| 595149|
|   2.0| 960915|
|   1.5| 186839|
|   1.0| 455078|
|   0.5| 162322|
+------+-------+



In [41]:
ratingsdf.describe().show()

+-------+------------------+------------------+------------------+--------------------+
|summary|            userId|           movieId|            rating|           timestamp|
+-------+------------------+------------------+------------------+--------------------+
|  count|          13406747|          13406747|          13406747|            13406747|
|   mean|46185.872029508726| 9062.768976023788|3.5249280828526115|1.1012320369401236E9|
| stddev|26835.497827007657|19800.549275374764| 1.052413058181816|1.6204168576153734E8|
|    min|                 1|                 1|               0.5|           789652004|
|    max|             92645|            131260|               5.0|          1427784002|
+-------+------------------+------------------+------------------+--------------------+



In [42]:
ratingsdf.describe("rating").show()

+-------+------------------+
|summary|            rating|
+-------+------------------+
|  count|          13406747|
|   mean|3.5249280828526115|
| stddev| 1.052413058181816|
|    min|               0.5|
|    max|               5.0|
+-------+------------------+



In [63]:
moviesdf.printSchema()
moviesdf.createTempView("title")

root
 |-- movieId: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)



In [67]:
spark.sql("select * from title limit 2").show()

+-------+----------------+--------------------+
|movieId|           title|              genres|
+-------+----------------+--------------------+
|      1|Toy Story (1995)|Adventure|Animati...|
|      2|  Jumanji (1995)|Adventure|Childre...|
+-------+----------------+--------------------+



In [60]:
ratingsdf.createOrReplaceTempView("rating") # inside the tempview is must be a column name of the data frame

In [61]:
spark.sql("select * from ratings order by rating desc limit 5").show()

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|   8507|   5.0|1094786027|
| 70568|    260|   5.0|1142392251|
| 35128|   5287|   5.0|1031583587|
| 70568|    527|   5.0|1142392243|
|     2|    266|   5.0| 974820748|
+------+-------+------+----------+



In [85]:
spark.sql('''select t.title,count(*) from title t join rating r on t.movieId=r.userId where t.genres='Comedy'
 group by t.title order by t.title desc limit 20''').show()

+--------------------+--------+
|               title|count(1)|
+--------------------+--------+
|Zorro, the Gay Bl...|    1088|
|    Zoolander (2001)|      24|
|    Zookeeper (2011)|     161|
|        Zelig (1983)|     222|
|Zeisters (Fat Guy...|      21|
|     Zebraman (2004)|      27|
|Zazie dans le mét...|     311|
|Yrrol: An Enormou...|      31|
|Young People Fuck...|      62|
|Young Einstein (1...|     561|
|Young Doctors in ...|     112|
|You, Me and Dupre...|      41|
|You're Telling Me...|      53|
|You Don't Mess wi...|     213|
|    You Again (2010)|      78|
|      Yolki 2 (2011)|      42|
|        Yolki (2010)|     218|
|Yesterday, Today ...|     176|
|      Yes Man (2008)|     339|
|         Xala (1975)|      66|
+--------------------+--------+

